# Applying Machine Learning To Sentiment Analysis
Notebook adopted from Sebastian Raschka's code examples from Machine Learning for Python

### Overview
In this ipynb we will expore sentiment analysis from two different approaches: as supervised classification task; and an unsupervised NLP analysis task.

In [ ]:
# !conda install --yes nltk

In [ ]:
import pandas as pd
import os

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer

import nltk
from nltk.stem.porter import PorterStemmer

# Toy Example to Introducing the bag-of-words model

The bag-of-words model is a representation of documents as vectors in a high dimensional vector space based on the training collection vocabulary. Its a way of representing text data when modeling text with machine learning algorithms.

The bag-of-words model is simple to understand and implement and has seen great success in problems such as language modeling and document classification.

It basically assumes an encoding whereby each word in the training dataset vocabulary is considered a feature. 

This means that when handling reviews from the test set we must also consider what happens to words that were not seen in the training set. Two ways to deal with unknown terms observed at test time are to have an unknown token that absorbs all cases of new vocabulary, or more commonly to simply ignore those words in the test collection.

## Transforming documents into feature vectors

By calling the fit_transform method on CountVectorizer, we just constructed the vocabulary of the bag-of-words model and transformed the following three sentences into sparse feature vectors:
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two


In [ ]:
count = CountVectorizer() #from sklearn.feature_extraction.text import CountVectorizer
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

Now let us print the contents of the vocabulary to get a better understanding of the underlying concepts:

In [ ]:
print(count.vocabulary_) # vocabulary_ attribute of CountVectorizer() shows a mapping of terms to feature indices.

As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary, which maps the unique words that are mapped to integer indices. Next let us print the feature vectors that we just created:

Each index position in the feature vectors shown here corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. For example, the  first feature at index position 0 resembles the count of the word "and", which only occurs in the last document, and the word is at index position 1 (the 2nd feature in the document vectors) occurs in all three sentences. Those values in the feature vectors are also called the raw term frequencies: *tf (t,d)*—the number of times a term t occurs in a document *d*.

We can print this as follows:

In [ ]:
print(bag.toarray())

# Exercise
Notice that in the above the CountVectorizer we created frquency counts for unigrams i.e. single words. 
To preserve some of the local ordering information we can also ask it to extract 2-grams of words in addition to the 1-grams (individual words). To do this modify the code as follows:

count = CountVectorizer(ngram_range=(1,2)) 


## Assessing word relevancy via term frequency-inverse document frequency

In [ ]:
np.set_printoptions(precision=2) # These options determine the way floating point numbers are displayed.

When we are analyzing text data, we often encounter words that occur across multiple documents from both classes. Those frequently occurring words typically don't contain useful or discriminatory information. In this subsection, we will learn about a useful technique called term frequency-inverse document frequency (tf-idf) that can be used to downweight those frequently occurring words in the feature vectors. The tf-idf can be de ned as the product of the term frequency and the inverse document frequency:

$$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

Here the tf(t, d) is the term frequency that we introduced in the previous section,
and the inverse document frequency *idf(t, d)* can be calculated as:

$$\text{idf}(t,d) = \text{log}\frac{N}{\text{df}(d, t)},$$

where $N$ is the total number of documents, and *df(d, t)* is the number of documents *d* that contain the term *t*. Note that adding the constant 1 to the denominator is optional and serves the purpose of assigning a non-zero value to terms that occur in all training samples; the log is used to ensure that low document frequencies are not given too much weight.

Scikit-learn implements yet another transformer, the `TfidfTransformer`, that takes the raw term frequencies from `CountVectorizer` as input and transforms them into tf-idfs:

In [ ]:
tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

As we saw in the previous subsection, the word "is" with the largest term frequency was contained in the 3rd document.

However, after transforming the same feature vector into tf-idfs, we see that the word "is" is
now associated with a relatively smaller tf-idf (0.45) in document 3 since it is
also contained in documents 1 and 2 and thus is unlikely to contain any useful, discriminatory information.

Note how "one" and "shining" are now considered more important even though they occured only once in the previous vector representation (using just TF).

However, if we'd manually calculated the tf-idfs of the individual terms in our feature vectors, we'd have noticed that the `TfidfTransformer` calculates the tf-idfs slightly differently compared to the standard textbook equations that we did in the lecture. The equations for the idf and tf-idf that were implemented in scikit-learn are:

The tf-idf equation that was implemented in scikit-learn is as follows:
$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

Here by setting smooth_idf=True, we acheive the extra addition of one to document frequencies, as if an extra document was seen containing every term in the collection exactly once. Prevents zero divisions.



While it is also more typical to normalize the raw term frequencies before calculating the tf-idfs, the `TfidfTransformer` normalizes the tf-idfs directly.

By default (`norm='l2'`), scikit-learn's TfidfTransformer applies the L2-normalization, which returns a vector of length 1 by dividing an un-normalized feature vector *v* by its L2-norm:

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$

To make sure that we understand how TfidfTransformer works, let us walk
through an example and calculate the tf-idf of the word is in the 3rd document.

The word "is" has a term frequency of 3 (tf = 3) in document 3, and the document frequency of this term is 3 since the term "is" occurs in all three documents (df = 3). Thus, we can calculate the idf as follows:

$$\text{idf}("is", d3) = log \frac{1+3}{1+3} = 0$$

Now in order to calculate the tf-idf, we simply need to add 1 to the inverse document frequency and multiply it by the term frequency:

$$\text{tf-idf}("is",d3)= 3 \times (0+1) = 3$$

In [ ]:
tf_is = 3 # suppose term "is" has a frequency of 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' % tfidf_is)

If we repeated these calculations for all terms in the 3rd document, we'd obtain the following tf-idf vectors: [3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]. However, we notice that the values in this feature vector are different from the values that we obtained from the TfidfTransformer that we used previously. The next step that we are missing in this tf-idf calculation is the L2-normalization, which can be applied as follows:

$$\text{tfi-df}_{norm} = \frac{[3.39, 3.0, 3.39, 1.29, 1.29, 1.29, 2.0 , 1.69, 1.29]}{\sqrt{[3.39^2+ 3.0^2+ 3.39^2+ 1.29^2+ 1.29^2+ 1.29^2+ 2.0^2+ 1.69^2+ 1.29^2]}}$$

$$=[0.5, 0.45, 0.5, 0.19, 0.19, 0.19, 0.3, 0.25, 0.19]$$

$$\Rightarrow \text{tfi-df}_{norm}("is", d3) = 0.45$$

lets recalculate the tfidf values by switching off the norm 

In [ ]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf 

In [ ]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

# TfidfVectorizer
Use TfidfVectorizer which is equivalent to CountVectorizer followed by TfidfTransformer.
It convert a collection of raw documents to a matrix of TF-IDF features.


In [ ]:
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
 ]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())

print(X.shape)

As we can see, the results match the results returned by scikit-learn's `TfidfTransformer` (Above). Since we now understand how tf-idfs are calculated, let us proceed to the next sections and apply those concepts to the movie review dataset.

# Exercise 
- Again try out the ngram_range=(1, 2, 3) to generate n-grams beyond ungrams with the TFIDFVectorizer.
- max_df and min_df are parameters that can be set with TFIDFVectorizer; where the former specifies the max cutoff for frequent occuring words and latter the minimum expected occurence of words with documents before words are considered. Try setting these and explore the different outputs. 

# Preparing the IMDb movie review data for text processing 

## Obtaining the IMDb movie review dataset

The IMDB movie review set can be downloaded from [http://ai.stanford.edu/~amaas/data/sentiment/](http://ai.stanford.edu/~amaas/data/sentiment/).
We have already done this and extracted the csv file. 

## Preprocessing the movie dataset into more convenient format

In [ ]:
df = pd.read_csv('data/movie_data_cat.csv', encoding='utf-8')
df.head(10)

In [ ]:
df.shape
df.columns

Since the sentiment column happens to be categorical we can map the "pos" and "neg" classes to 0 and 1 integers. 

In [ ]:
class_mapping = {label:idx for idx,label in enumerate(np.unique(df['sentiment']))}

print(class_mapping)

#use the mapping dictionary to transform the class labels into integers

df['sentiment'] = df['sentiment'].map(class_mapping)
df.head(10)

Now that the class column is as we need it for a classifier we next look at how to clean up the review text content. 

## Cleaning text data with Regular Expressions
execute the code below to view the first review. 
You will notice that the text needs cleaned up e.g. due to html markup, punctuation and other non-letter chars. 

In [ ]:
df.loc[5635, 'review']#[-50:]

we can use python's regular expression library to clean up some of this data.
for details on the re library goto : https://docs.python.org/2/library/re.html

In [ ]:
#import regular expressions to clean up the text
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # remove all html markup
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text) # findall the emoticons
    
    # remove the non-word chars '[\W]+'
    # append the emoticons to end 
    #convert all to lowercase
    # remove nose char for consistency
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', '')) 
    return text

In [ ]:
preprocessor(df.loc[3635, 'review'])#[-50:]

## Apply the clean data preprocessor to the text

In [ ]:
preprocessor("</a>This :) is :( a test :-)!")

In [ ]:
# apply the preprocessor to the entire dataframe (i.e. column review)
df['review'] = df['review'].apply(preprocessor)

## Tokenise - break text into tokens

In [ ]:
def tokenizer(text):
       return text.split()


In [ ]:
print(tokenizer("Tokenise this sentence into its individual words"))

## Stopwords - Removing stopwords from text
We need todown load the stopwords list from nltk.
You can do that as follows:


In [ ]:
from nltk.corpus import stopwords 

nltk.download('stopwords')

create a method to accept a piece of tokenised text and return text back without the stopped words

In [ ]:
stop = set(stopwords.words('english'))
def stop_removal(text):
       return [w for w in text if not w in stop]

In [ ]:
text = "This is a sample sentence, demonstrating the removal of stop words."
stopped_text = stop_removal(text.split())
print(stopped_text) 

## Stemming - Processing tokens into their root form
For this purpose we will explore two different stemmers and select one.

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

#See which languages are supported.
print(" ".join(SnowballStemmer.languages))

In [ ]:
#get the english stemmer
stemmer = SnowballStemmer("english")

#stem a word
print(stemmer.stem("running"))

In [ ]:
#Decide not to stem stopwords with ignore_stopwords
stemmer2 = SnowballStemmer("english", ignore_stopwords=True)

#compare the two versions of the stemmer
print(stemmer.stem("having"))

print(stemmer2.stem("having"))

In [ ]:
#The 'english' stemmer is better than the original 'porter' stemmer.
print(SnowballStemmer("english").stem("generously"))

print(SnowballStemmer("porter").stem("generously"))

# Tokenise + Stemming 
Lets create  method to stem each word / token contained in the piece of text. 
Note how the text is first tokenised before stemming.

In [ ]:
def tokenizer_stemmer(text):
    return [stemmer.stem(word) for word in tokenizer(text)]#text.split()]

In [ ]:
tokenizer('runners like running and thus they run')

In [ ]:
tokenizer_stemmer('runners like running and thus they run')

You can clearly see from the code above the effect of the stemmer on the tokens

In [ ]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_stemmer('A runner likes running and runs a lot')[-8:]
if w.lower() not in stop]


# Training a model for sentiment classification

Strip HTML and punctuation to speed up the GridSearch later:

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

### smaller sample
X_train = df.loc[:2500, 'review'].values
y_train = df.loc[:2500, 'sentiment'].values

In [ ]:
param_grid0 = [{'vect__ngram_range': [(1, 1)], #can also extract 2-grams of words in addition to the 1-grams (individual words)
               'vect__stop_words': [stop, None], # use the stop dictionary of stopwords or not
               'vect__tokenizer': [tokenizer_stemmer]}, # use a tokeniser and the stemmer 
               ]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV



tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)


mnb_tfidf = Pipeline([('vect', tfidf),
                     ('clf',  MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])


                   
gs_mnb_tfidf = GridSearchCV(mnb_tfidf, param_grid0,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=1) 

**Important Note about the running time**

Executing the following code cell **may take up to 30-60 min** depending on your machine, since based on the parameter grid we defined, there are 2*2*2*3*5 + 2*2*2*3*5 = 240 models to fit.

If you do not wish to wait so long, you could reduce the size of the dataset by decreasing the number of training samples, for example, as follows:

    X_train = df.loc[:2500, 'review'].values
    y_train = df.loc[:2500, 'sentiment'].values
    
However, note that decreasing the training set size to such a small number will likely result in poorly performing models. Alternatively, you can delete parameters from the grid above to reduce the number of models to fit -- for example, by using the following:

    param_grid = [{'vect__ngram_range': [(1, 1)],
                   'vect__stop_words': [stop, None],
                   'vect__tokenizer': [tokenizer],
                  ]

In [ ]:
gs_mnb_tfidf.fit(X_train, y_train)

In [ ]:
print('Best parameter set: %s ' % gs_mnb_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_mnb_tfidf.best_score_)

In [ ]:
clf = gs_mnb_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

####  GridSearchCV versus cross_val_score
    
Please note that `gs_mnb_tfidf.best_score_` is the average k-fold cross-validation score. I.e., if we have a `GridSearchCV` object with 5-fold cross-validation (like the one above), the `best_score_` attribute returns the average score over the 5-folds of the best model. To illustrate this with an example:

# Exercise
In the previous code you saw that having the stopword remover (i.e. stop ) was better than None (not having stopword removal). You can now explore other alternatives. Change the param_grid0  values to explore the folloing:
- Does having Tokenizer_stemmer result in better accuracy compared to just the tokenizer (with no stemmer)?
- What happens if you consider bith unigrams and bigrams? Is there a benfit to analysing bigrams?
- How can you modify the classifier 'clf' to a different learner such as the MLPClassifier or anyone the learners you have tried in the previous lab.

# Unsupervised Sentiment Analysis
In the previous code we treated Sentimetn Analysis as a standard text classification problem. 
In this section we will adopt a different approach; whereby we will make use of a sentiment dictionary (SentWornNet)
to score every word in a piece of text as either positive or negative. 
Instead of fitting a machine learning classifier; we will instead aggregate these scores to establish the overall poloarity of the piece of text. 

In [ ]:
# lets download the sentiwordnet lexicon / dictionary from NLTK
# we also need to the original wordnet lexicon 
nltk.download('sentiwordnet')
nltk.download('wordnet')

## Using the sentiwordnet Lexicon

In [ ]:
from nltk.corpus import sentiwordnet as swn

awesome = list(swn.senti_synsets('awesome', 'a'))[0]
print('Positive Polarity Score:', awesome.pos_score())
print('Negative Polarity Score:', awesome.neg_score())
print('Objective Score:', awesome.obj_score())

## Exercise
Using the cell above try SentiWordNet to explore the pos, neg scores for different words.
Note that 'a' refers to adjectives. If you are looking at anoun then this should be set to 'n' and similarly for verbs ('v') and adverbs ('r). 

# POS Tagging - Setup NLTK for pos tagging

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

Unlike the previous tokenizer method we had created with the simple split function we now need to both tokenise and associate the relevant part-of-speech (POS) tag to each word. To do this we will make use the of nltk.pos_tag as shown in the example below, 

In [ ]:
tagged_text =nltk.pos_tag(tokenizer('The cat sat on the mat.'))
tagged_text


We can modify the code above such that it returns a list.
This data construct is more convinient as we see in the next cell using word_tokenise from the nltk library.

## Unsupervised Sentiment Analyser

In [ ]:
from nltk import word_tokenize

def unsupervised_sentiment_analyzer(review, verbose=False):

    # tokenize and POS tag text tokens
    #tagged_text = [(token.text, token.tag_) for token in nltk.pos_tag tn.nlp(review)]
    
    tagged_text = [(pair) for pair in nltk.pos_tag(word_tokenize(review))]
    pos_score = neg_score = token_count = obj_score = 0
    # get wordnet synsets based on POS tags
    # get sentiment scores if synsets are found
    for word, tag in tagged_text:
        ss_set = None
        if 'NN' in tag and list(swn.senti_synsets(word, 'n')): #noun
            ss_set = list(swn.senti_synsets(word, 'n'))[0]
        elif 'VB' in tag and list(swn.senti_synsets(word, 'v')): #verb
            ss_set = list(swn.senti_synsets(word, 'v'))[0]
        elif 'JJ' in tag and list(swn.senti_synsets(word, 'a')): #adjective
            ss_set = list(swn.senti_synsets(word, 'a'))[0]
        elif 'RB' in tag and list(swn.senti_synsets(word, 'r')): #adverb
            ss_set = list(swn.senti_synsets(word, 'r'))[0]
        # if senti-synset is found        
        if ss_set:
            # add scores for all found synsets
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    
    # aggregate final scores
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = 'positive' if norm_final_score >= 0 else 'negative'
    if verbose:
        norm_obj_score = round(float(obj_score) / token_count, 2)
        norm_pos_score = round(float(pos_score) / token_count, 2)
        norm_neg_score = round(float(neg_score) / token_count, 2)
        # to display results in a nice table
        sentiment_frame = pd.DataFrame([[final_sentiment, norm_obj_score, norm_pos_score, 
                                         norm_neg_score, norm_final_score]],
                                       columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                             ['Predicted Sentiment', 'Objectivity',
                                                              'Positive', 'Negative', 'Overall']], 
                                                             labels=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    return 1 if (final_sentiment == 'positive') else 0
    #return final_sentiment

## Predict Sentiment for some example text

In [ ]:
unsupervised_sentiment_analyzer("stupid movie, acting average or worse... screenplay - no sense at all... SKIP IT!", verbose=True)

# Exercise
You can see that the prediction is 0 which in this case relates to non-positive sentiment class i.e. negative sentiment.
Modify the piece of text above so that the sentiment analyser predicts positive class. 

## Predict Sentiment for a selected movie review
You can use the code below to explore the prediction given a specific id of a review.
In the example we have used '5635'. Try out different reviews ids. 

In [ ]:
print(df.loc[5635, 'review'])
pred = unsupervised_sentiment_analyzer(df.loc[5635, 'review'], verbose=True)
print('predicted:', pred)

# Evaluate Sentiment Prediction on the Movie Review Test Dataset¶


In [ ]:
#Choose the test sample
reviews = X_test #df.loc[25000:25003,['review']].values
sentiments = y_test #df.loc[25000:25003,['sentiment']].values
#The original test dataset has 25000 
# This may take considerable time on a standard machine
# Ideally select a sample to test with
reviews = df.loc[25000:26000,'review'].values
sentiments = df.loc[25000:26000,'sentiment'].values

count = 0
correct = 0

msg = "predicting for %d reviews" % len(reviews)
print(msg)
print('this will take a moment ...')
for test_X, test_y in zip(reviews, sentiments):#zip(df['review'], df['sentiment']):
    count+=1
    #print('REVIEW:', review)
    #print('Actual Sentiment:', test_y)
    pred = unsupervised_sentiment_analyzer(test_X, verbose=False) 
    #print('predicted Sentiment:', pred)
    #print('-'*60)
    if (pred==test_y):
        correct+=1

accuracy = round(float(correct) / count, 3) * 100
print(accuracy)
print(correct, count)


# Exercise
- Which approach results in better accuracy; supervised or unsupervised sentiment analysis?
- What do you think might be the pros and cons of the 2 approaches to sentment analysis i.e. using a text classification or supervised approach versus using the -precompiled dictionary of sentment knowledge to analyse text in an unsupervised setting?